# Libraries

In [ ]:
# 📚 Basic Libraries
import pandas as pd
import numpy as np
import warnings
import os
import plotly.express as px
import zipfile
from google.colab import files

# 📊 Visualizations
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

# 01 | About Dataset

[Kaggle link](https://www.kaggle.com/datasets/sadiqshah/bike-sales-in-europe/data)  

**Columns:**
- `Date`: The date of the sale.
- `Day`: The day of the month when the sale occurred.
- `Month`: The month when the sale occurred.
- `Year`: The year when the sale occurred.
- `Customer_Age`: Age of the customer.
- `Age_Group`: Age group classification of the customer.
- `Customer_Gender`: Gender of the customer.
- `Country`: Country where the sale took place.
- `State`: State where the sale took place.
- `Product_Category`: Category of the product sold.
- `Sub_Category`: Sub-category of the product sold.
- `Product`: Specific product sold.
- `Order_Quantity`: Number of units ordered.
- `Unit_Cost`: Cost per unit of the product.
- `Unit_Price`: Selling price per unit of the product.
- `Profit`: Profit from the sale.
- `Cost`: Total cost of the sale.
- `Revenue`: Total revenue from the sale.

**Problem Statement**

**Business Analyst Case:** A bike shop has been gathering data from 2011-2016 and now wants to gain insights and understand historical performance to improve the business. The objective is to use this data to predict future demand and profits, which are key for making informed business decisions.


# Extract Data

In [ ]:
os.listdir()

['.config',
 'road_bikes.csv',
 '2016_cleaned_data.csv',
 'mountain_bikes.csv',
 'touring_bikes.csv',
 'sample_data']

In [ ]:
df_2016 = pd.read_csv('2016_cleaned_data.csv')  # 2016 all data
road_bikes_df = pd.read_csv('road_bikes.csv')
mountain_bikes_df = pd.read_csv('mountain_bikes.csv')
touring_bikes_df = pd.read_csv('touring_bikes.csv')

# 04 | Streamlit App

In [ ]:
! pip install streamlit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00


In [ ]:
# Write Streamlit App:
%%writefile app.py
import streamlit as st
import pandas as pd

# Load the DataFrames:
df_2016 = pd.read_csv('2016_cleaned_data.csv') # 2016 all data
road_bikes_df = pd.read_csv('road_bikes.csv')
mountain_bikes_df = pd.read_csv('mountain_bikes.csv')
touring_bikes_df = pd.read_csv('touring_bikes.csv')

# App title and logo:
st.title('Bike Shop App')
st.image("https://www.shutterstock.com/image-vector/bike-icon-vector-logo-template-600nw-1388480312.jpg", width=350)

# App tabs:
tabs = st.tabs(["Product Price Catalog", "Bike Budget Planner"])

# First App tab [Product Price Catalog]:
with tabs[0]:
    st.header('Product Price Catalog')
    st.write("Select a product and consult product's manufacturing cost and final price:")

    # Create Select Boxes:

    # 1) Product category
    product_category = st.selectbox('Select Product Category:', df_2016['product_category'].unique())

    # Filter dataframe based on selected category:
    filtered_product_df = df_2016[df_2016['product_category'] == product_category]

    # 2) Sub Category
    sub_category = st.selectbox('Select Product Sub-Category:', filtered_product_df.sub_category.unique())

    # Filter again dataframe based on selected sub_category:
    filtered_sub_category_df = filtered_product_df[filtered_product_df['sub_category'] == sub_category]

    # 3) Product list:
    product = st.selectbox('Select Product:', filtered_sub_category_df['product'].unique())

    # Function to get unit cost and price
    def product_unit_cost_price(df, product):
        """Pulls unit cost and price of a given product"""
        filtered_df = df[df['product'] == product]
        if not filtered_df.empty:
            unit_cost = filtered_df['unit_cost'].unique()[0]
            unit_price = filtered_df['unit_price'].unique()[0]
            return unit_cost, unit_price
        else:
            return None, None

    # Call the function with the selected product
    unit_cost, unit_price = product_unit_cost_price(filtered_sub_category_df, product)

    # Display the result in the Streamlit app
    if unit_cost is not None and unit_price is not None:
        # Calculate the profit margin based on selling price
        profit_margin = ((unit_price - unit_cost) / unit_price) * 100

        st.write('Results of the selected product:')
        st.write(f"Unit Cost: {unit_cost:.2f}€")
        st.write(f"Unit Price: {unit_price:.2f}€")
        st.write(f"Profit Margin: {profit_margin:.2f}%")
        st.write(f'--------------------------------')
    else:
        st.write("No matching product found, review the product name")

# -------------------------------------- #

# Second App tab [Bike Budget Planner]:
with tabs[1]:
    st.header('Bike Budget Planner')
    st.write("Select a bike category, choose a budget range, and consult potential products:")

    # 1) Bike Category Selector:
    category = st.selectbox('Select Product Category:', ['Road Bike', 'Mountain Bike', 'Touring Bike'])

    # 2) Double-Ended Budget Slider:
    budget_range = st.slider('Select Budget Range:', min_value=500, max_value=5000, value=(1000, 3000), step=100)

    # 3) Determine DataFrame based on the selected category:
    if category == 'Road Bike':
        df = road_bikes_df
    elif category == 'Mountain Bike':
        df = mountain_bikes_df
    elif category == 'Touring Bike':
        df = touring_bikes_df

    # Ensure that the DataFrame is loaded and contains the expected columns:
    if 'unit_price' in df.columns and 'product' in df.columns:
        # Debugging output:
        st.write(f"Selected category: {category}")
        st.write(f"Selected budget range: {budget_range[0]}€ - {budget_range[1]}€")
        st.write(f"DataFrame shape: {df.shape}")
        st.write(f"Unit price range: {df['unit_price'].min()}€ - {df['unit_price'].max()}€")

        # 4) Filter DataFrame based on budget range:
        filtered_budget_df = df[(df['unit_price'] >= budget_range[0]) & (df['unit_price'] <= budget_range[1])]

        # Display results:
        if not filtered_budget_df.empty:
            st.write(f"Products within your budget range of {budget_range[0]}€ to {budget_range[1]}€:")
            st.write(filtered_budget_df[['product', 'unit_cost', 'unit_price', 'profit_margin']])
        else:
            st.write("No products found within the selected budget range.")
    else:
        st.write("The selected category does not have the expected data.")

Writing app.py


In [ ]:
# Run the Streamlit App:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.221.37.42:8501

Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) y
your url is: https://flat-birds-accept.loca.lt
  Stopping...
^C
